In [16]:
import pandas as pd
import os
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import json

from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import re

In [24]:
test_df.columns = ['text','type','summary','type_code']
test_df = test_df.reset_index(drop = True)
test_df.head()
test_df.to_csv('../data/temp/test.csv', index=False)

In [25]:
test_df.shape[0]

50

In [133]:
data_folder = "../data/raw/BBC News Summary/News Articles/"
entries = os.listdir(data_folder)
entries

['business', 'entertainment', 'politics', 'sport', 'tech']

In [131]:
all_data = {}
for i in entries:
    temp = []
    folder_path = data_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_data[i] = temp

In [ ]:
summary_folder = "../data/raw/BBC News Summary/Summaries/"
summary_entries = os.listdir(data_folder)
all_sum = {}
for i in entries:
    temp = []
    folder_path = summary_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_sum[i] = temp

In [21]:
total_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in np.arange(len(entries)):
    if i == 0:
        total_df = pd.DataFrame.from_dict(all_data['business'])
        total_df['type'] = 'business'
        total_df['summary'] = all_sum['business']
        total_df['type_code'] = i+1
        
        
        index = np.random.choice(total_df.shape[0],10)
        test_df = total_df.loc[index]
    else:
        temp_df = pd.DataFrame.from_dict(all_data[entries[i]])
        temp_df['type'] = entries[i]
        temp_df['summary'] = all_sum[entries[i]]
        temp_df['type_code'] = i+1
        total_df =pd.concat([total_df, temp_df], axis=0)
        
        index = np.random.choice(temp_df.shape[0], 10)
        temp_test_df = temp_df.loc[index]
        test_df =pd.concat([test_df,temp_test_df] , axis=0) 
        
total_df.columns = ['text','type','summary','type_code']
total_df = total_df.reset_index(drop = True)
total_df.to_csv('../data/temp/all_data.csv', index=False)

In [22]:
total_df.shape[0]

2225

In [26]:
data_folder = "../data/raw/BBC News Summary/News Articles/"
entries = os.listdir('../data/raw/BBC News Summary/News Articles/')

In [32]:
temp = "../data/raw/BBC News Summary"

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [5]:
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T


In [12]:
def regex_condi(string):
    if ".com" in string:
        return False
    if ".edu" in string:
        return False
    if "@" in string:
        return False
    if 'Host'in string:
        return False
    if ".gov" in string:
        return False

    return True


def clean_text(string):
    string = " ".join([i for i in string.split() if regex_condi(i)])
    string = re.sub(r"Re:","", string)
    string = re.sub(r"Reply-To:","", string)

    string_lst = string.split(': ')
    
    
    toreturn = ""
    for i in np.arange(len(string_lst) - 1):
        if "Subject" in string_lst[i]:
            temp = string_lst[i+1].split(' ')
            toreturn += " " + ' '.join(temp[:-1])
        if "Keywords" in string_lst[i]:
            temp = string_lst[i+1].split(' ')
            toreturn += " " + ' '.join(temp[:-1])
        if "Organization" in string_lst[i]:
            temp = string_lst[i+1].split(' ')
            toreturn += " " + ' '.join(temp[:-1])
            
        if "Lines" in string_lst[i]:
            temp = ""
            for j in range(i+1,len(string_lst)):
                temp += string_lst[j]
            toreturn += temp
            break

    toreturn = re.sub(r"\'", "", toreturn)
    toreturn = re.sub(r"\>", "", toreturn)
    toreturn = re.sub(r"\:", "", toreturn)
    toreturn = re.sub(r"^[0-9]+ ", "", toreturn)
    toreturn = re.sub(r"\/", ' ',toreturn)
    toreturn = re.sub(r"\-{2,}", ' ',toreturn)
    toreturn = re.sub(r"\s+", ' ',toreturn)
    toreturn = toreturn.strip()
    return toreturn

In [13]:
os.makedirs('../data/temp',exist_ok=True)

In [30]:
tar_0 = news_df[news_df.target == 7]

In [31]:
with open('../data/temp/tar_0.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(tar_0.data.apply(lambda x: clean_text(x)+ '.')))

In [32]:
para = ' '.join([f'{param}={value}' for param, value in json.load(open('../config/autophrase_cfg.json')).items()])

In [17]:
os.system('git submodule init && git submodule update')

0

In [18]:
os.system(f'cd ../AutoPhrase/ && {para} ./auto_phrase.sh')

0

In [16]:
news_df.data = news_df.data.apply(lambda x: clean_text(x)+ '.')

In [39]:
tar_0.data.apply(lambda x: len(x.split(' ')))

0         114
17       1574
29        108
56        360
64         82
         ... 
11238     340
11282     386
11288     227
11291     232
11292     352
Name: data, Length: 594, dtype: int64

In [35]:
news_df

,data,filenames,target_names,target,DESCR
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,/home/yal319/scikit_learn_data/20news_home/20n...,alt.atheism,7,.
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.graphics,4,.
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,4,
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.ibm.pc.hardware,1,_
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.mac.hardware,14,2
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,/home/yal319/scikit_learn_data/20news_home/20n...,None,13,None
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,/home/yal319/scikit_learn_data/20news_home/20n...,None,4,None
11311,From: westes@netcom.com (Will Estes)\nSubject:...,/home/yal319/scikit_learn_data/20news_home/20n...,None,3,None
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,/home/yal319/scikit_learn_data/20news_home/20n...,None,1,None


In [34]:
news_df.target.value_counts()

10    600
15    599
8     598
9     597
11    595
7     594
13    594
5     593
14    593
12    591
2     591
3     590
6     585
1     584
4     578
17    564
16    546
0     480
18    465
19    377
Name: target, dtype: int64

In [134]:
data_folder = "../data/raw/20_newsgroups/"
entries = os.listdir(data_folder)
all_data = {}
for i in entries:
    temp = []
    folder_path = data_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_data[i] = temp

In [137]:
total_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in np.arange(len(entries)):
    if i == 0:
        total_df = pd.DataFrame.from_dict(all_data['alt.atheism'])
        total_df['type'] = 'alt.atheism'
        total_df['type_code'] = i+1
        
        
        index = np.random.choice(total_df.shape[0],10)
        test_df = total_df.loc[index]
    else:
        temp_df = pd.DataFrame.from_dict(all_data[entries[i]])
        temp_df['type'] = entries[i]
        temp_df['type_code'] = i+1
        total_df = pd.concat([total_df, temp_df], axis=0)
        
        index = np.random.choice(temp_df.shape[0], 10)
        temp_test_df = temp_df.loc[index]
        test_df =pd.concat([test_df,temp_test_df] , axis=0) 
        

total_df = total_df.reset_index(drop = True)


In [ ]:
total_df.to_csv('../data/temp/all_data.csv', index=False)

In [41]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [42]:
newsgroups_test = fetch_20newsgroups(subset='test')

In [45]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(clean_data)
count_vect.vocabulary_.get(u'algorithm')

19127

In [44]:
clean_data = [clean_text(i) for i in newsgroups_train.data]

In [46]:

text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('clf', LogisticRegression(max_iter = 100))
 ])
text_clf.fit(clean_data, newsgroups_train.target)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('clf', LogisticRegression())])

In [50]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test')
docs_test = [clean_text(i) for i in twenty_test.data]
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.7616834838024429

In [51]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.65      0.66      0.65       319
           comp.graphics       0.67      0.73      0.70       389
 comp.os.ms-windows.misc       0.73      0.64      0.68       394
comp.sys.ibm.pc.hardware       0.63      0.64      0.64       392
   comp.sys.mac.hardware       0.76      0.77      0.77       385
          comp.windows.x       0.81      0.71      0.76       395
            misc.forsale       0.79      0.86      0.82       390
               rec.autos       0.78      0.81      0.80       396
         rec.motorcycles       0.85      0.90      0.87       398
      rec.sport.baseball       0.82      0.88      0.85       397
        rec.sport.hockey       0.91      0.90      0.91       399
               sci.crypt       0.89      0.84      0.87       396
         sci.electronics       0.66      0.69      0.67       393
                 sci.med       0.81      0.77      0.79       396
         

In [52]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 120548)

In [53]:
text_clf1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])
text_clf1.fit(newsgroups_train.data, newsgroups_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression())])

In [54]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test')
docs_test = twenty_test.data
predicted = text_clf1.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8274030801911842

In [55]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.74      0.77       319
           comp.graphics       0.69      0.79      0.74       389
 comp.os.ms-windows.misc       0.75      0.73      0.74       394
comp.sys.ibm.pc.hardware       0.72      0.72      0.72       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.76      0.90      0.82       390
               rec.autos       0.90      0.89      0.90       396
         rec.motorcycles       0.95      0.95      0.95       398
      rec.sport.baseball       0.88      0.92      0.90       397
        rec.sport.hockey       0.94      0.95      0.95       399
               sci.crypt       0.94      0.88      0.91       396
         sci.electronics       0.76      0.80      0.78       393
                 sci.med       0.89      0.83      0.85       396
         

In [72]:
twenty_test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [53]:
news_df.head(20)

,data,filenames,target_names,target,DESCR
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,/home/yal319/scikit_learn_data/20news_home/20n...,alt.atheism,7,.
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.graphics,4,.
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,4,
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.ibm.pc.hardware,1,_
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.mac.hardware,14,2
5,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.windows.x,16,0
6,From: bmdelane@quads.uchicago.edu (brian manni...,/home/yal319/scikit_learn_data/20news_home/20n...,misc.forsale,13,n
7,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,/home/yal319/scikit_learn_data/20news_home/20n...,rec.autos,3,e
8,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,/home/yal319/scikit_learn_data/20news_home/20n...,rec.motorcycles,2,w
9,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,/home/yal319/scikit_learn_data/20news_home/20n...,rec.sport.baseball,4,s


In [24]:
df = pd.read_csv('../model/20news/AutoPhrase.txt',error_bad_lines=False, delimiter= '\t',names = ['p','phrase'])
select = df[df.p >= 0.5]

In [25]:
text_for_vect = ' '.join(select.phrase.apply(lambda x: str(x)))

In [26]:
count_vect = CountVectorizer(vocabulary=select.phrase ,stop_words = 'english')
count_vect = count_vect.fit([text_for_vect])
len(count_vect.vocabulary_)

8144

In [27]:
X_count = count_vect.transform(news_df.data)

In [144]:
clf = LogisticRegression(max_iter = 100).fit(X_count,newsgroups_train.target)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [145]:
twenty_test = fetch_20newsgroups(subset='test')
docs_test = count_vect.transform(twenty_test.data)
predicted = clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.778146574614976

In [149]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_count)
X_train_tfidf.shape

(11314, 8148)

In [150]:
clf = LogisticRegression(max_iter = 150).fit(X_train_tfidf,newsgroups_train.target)

In [151]:
twenty_test = fetch_20newsgroups(subset='test')
docs_test = count_vect.transform(twenty_test.data)
docs_test = tfidf_transformer.transform(docs_test)
predicted = clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8100106213489113

In [152]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.74      0.69      0.72       319
           comp.graphics       0.71      0.78      0.74       389
 comp.os.ms-windows.misc       0.74      0.74      0.74       394
comp.sys.ibm.pc.hardware       0.67      0.70      0.68       392
   comp.sys.mac.hardware       0.80      0.77      0.78       385
          comp.windows.x       0.81      0.74      0.77       395
            misc.forsale       0.78      0.85      0.81       390
               rec.autos       0.88      0.88      0.88       396
         rec.motorcycles       0.90      0.93      0.92       398
      rec.sport.baseball       0.92      0.90      0.91       397
        rec.sport.hockey       0.93      0.96      0.95       399
               sci.crypt       0.95      0.89      0.92       396
         sci.electronics       0.70      0.73      0.71       393
                 sci.med       0.87      0.85      0.86       396
         

In [42]:
count_vect = CountVectorizer(vocabulary=select.phrase ,stop_words = 'english')
count_vect = count_vect.fit([text_for_vect])
len(count_vect.vocabulary_)

8144

In [43]:
X_count = count_vect.transform(tar_0.data)

In [52]:
X_count.toarray().sum(axis = 1)

array([  48,  599,   51,  152,   44,  112,   63,   80,   64,  141,   49,
         55,   33,  161,   45,   44,   77,  115,   19,   47,   86,   92,
         83,   64,   40,   45,   30,   71,   85,  156,  167,   41,  207,
         30,  131,   50,   39,   95,   29,   90,  200,  231,   79,    9,
         62,   65,   90,  263,   61,   65,  147,   84,  197,   70,   56,
        128,   53,  124,   25,  143,   52,   30,   84,   91,  219,   34,
         67,  119,  143,  255,  111,   19,  764,   54,   83,   30,  107,
         62,   57,  122,   52,  104,   88,   42,   88,  128,  192,   88,
        105,   53,  107,  103,   32,   96,  293,   82,  128,  216,   27,
         67,   25,   57,   53,   40,   91,   42,   19,   89,   78,   42,
         70,   69,   50,   40,   45,  119,   95,   48,   40,   43,  107,
         44,   48,  101,  269,   29,   62,   77,   31,  116,  127,   12,
         41,   49,   64,   27,  101,   70,   45,   55,   24,   59,   44,
        384,   50,  107,   50,   63,  151,  133,  1

In [54]:
news_df

,data,filenames,target_names,target,DESCR
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,/home/yal319/scikit_learn_data/20news_home/20n...,alt.atheism,7,.
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.graphics,4,.
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,4,
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.ibm.pc.hardware,1,_
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,/home/yal319/scikit_learn_data/20news_home/20n...,comp.sys.mac.hardware,14,2
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,/home/yal319/scikit_learn_data/20news_home/20n...,None,13,None
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,/home/yal319/scikit_learn_data/20news_home/20n...,None,4,None
11311,From: westes@netcom.com (Will Estes)\nSubject:...,/home/yal319/scikit_learn_data/20news_home/20n...,None,3,None
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,/home/yal319/scikit_learn_data/20news_home/20n...,None,1,None


In [163]:
from sklearn.neural_network import MLPClassifier

In [169]:
%%time
clf = MLPClassifier(random_state=1, max_iter=100).fit(X_train_tfidf,newsgroups_train.target)

CPU times: user 19min 58s, sys: 2h 59min 27s, total: 3h 19min 25s
Wall time: 26min 3s


/opt/conda/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [171]:
twenty_test = fetch_20newsgroups(subset='test')
docs_test = count_vect.transform(twenty_test.data)
docs_test = tfidf_transformer.transform(docs_test)
predicted = clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8193043016463091